# 02 - Spectrogram Generation
## Converting IQ Data to RF Spectrograms

This notebook covers:
- Generating spectrograms from IQ data
- Georgia Tech normalization procedure
- Batch processing multiple files
- Saving spectrograms as PNG images

In [ ]:
# Setup
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import cv2

from src.io_utils import load_iq_data
from src.spectrogram import generate_spectrogram
from src.preprocessing import process_iq_files_to_spectrograms

print("✅ All imports successful!")

## Configuration

In [ ]:
# Paths
RAW_DATA_DIR = Path('../data/raw')
SPECTROGRAM_DIR = Path('../data/spectrograms')
SPECTROGRAM_DIR.mkdir(parents=True, exist_ok=True)

# Parameters
SAMPLE_RATE = 20e6  # 20 MHz
SEGMENT_DURATION = 410e-6  # 410 μs per frame
DTYPE = 'sc16'
COLORMAP = 'viridis'  # Options: 'viridis', 'plasma', 'turbo', 'jet'

print(f"Sample rate: {SAMPLE_RATE/1e6} MHz")
print(f"Segment duration: {SEGMENT_DURATION*1e6} μs")
print(f"Samples per segment: {int(SEGMENT_DURATION * SAMPLE_RATE)}")

## Generate Single Spectrogram (Test)

In [ ]:
# Load sample IQ data
iq_files = sorted(RAW_DATA_DIR.glob(f'*.{DTYPE}'))

if len(iq_files) > 0:
    sample_file = iq_files[0]
    iq_data = load_iq_data(sample_file, dtype=DTYPE)
    
    # Take first segment
    samples_per_segment = int(SEGMENT_DURATION * SAMPLE_RATE)
    segment = iq_data[:samples_per_segment]
    
    # Generate spectrogram
    spec_img, t, f = generate_spectrogram(
        segment,
        fs=SAMPLE_RATE,
        colormap_name=COLORMAP
    )
    
    print(f"Spectrogram shape: {spec_img.shape}")
    print(f"Time axis: {len(t)} points")
    print(f"Frequency axis: {len(f)} points")
    
    # Display
    plt.figure(figsize=(10, 8))
    plt.imshow(spec_img, aspect='auto', origin='lower')
    plt.title('Sample Spectrogram')
    plt.xlabel('Time')
    plt.ylabel('Frequency')
    plt.colorbar(label='Intensity')
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No IQ files found")

## Batch Process All IQ Files

In [ ]:
# Process all files in the raw directory
process_iq_files_to_spectrograms(
    input_dir=RAW_DATA_DIR,
    output_dir=SPECTROGRAM_DIR,
    file_pattern=f'*.{DTYPE}',
    segment_duration=SEGMENT_DURATION,
    fs=SAMPLE_RATE,
    dtype=DTYPE,
    colormap=COLORMAP
)

## Verify Generated Spectrograms

In [ ]:
# List generated spectrograms
spec_files = sorted(SPECTROGRAM_DIR.glob('*.png'))
print(f"Generated {len(spec_files)} spectrograms")

# Display first few
if len(spec_files) > 0:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for idx, spec_file in enumerate(spec_files[:6]):
        img = cv2.imread(str(spec_file))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        axes[idx].imshow(img_rgb)
        axes[idx].set_title(spec_file.stem)
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

## Next Steps

Continue to notebook 03 for sliding window analysis.